# Lab 06: Spatial smoothing, denoising with ICA, and temporal filtering

The goal of this lab is to practice common pre-preprocessing steps that all involve some form of filtering out signal that is deemed to be "noise" based on signal processing concepts. We will look at three common steps that follow motion correction and slice-timing: 
* Spatial smoothing
* ICA-based denoising
* Temporal filtering

## Preparation

In [1]:
%%bash

# DECIDE WHETHER YOU WANT TO COPY (cp) OR DOWNLOAD THE FILES NECESSARY FOR THIS LAB

# default
ans="dl"

if [ "${ans}" == "cp" ]; then
    cp ../05-Lab/my_bold_mcf_st.nii.gz my_bold_mcf_st.nii.gz
    cp ../05-Lab/my_bold_mcf.par my_bold_mcf.par
    cp ../04-Lab/func2T1.mat func2T1.mat
    cp ../04-Lab/T1_2_MNI_warp.nii.gz T1_2_MNI_warp.nii.gz
    # need some other generated outputs from ICA-AROMA
elif [ "${ans}" == "dl" ]; then
    wget --quiet -O 06-Lab_data.tar.gz https://osf.io/kf43u/download &&\
    tar -xf 06-Lab_data.tar.gz &&\
    rm 06-Lab_data.tar.gz
else
    printf 'YOU FORGOT TO CHANGE ANS TO "cp" OR "dl"'
fi

bash: fg: %%bash: no such job


## Spatial smoothing in FSL

FSL's smoothing tool is called SUSAN
* See user manual [here](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/SUSAN)
* An important usage note is that SUSAN takes in the smoothing kernel as "sigma" which refers to the standard deviation of the smoothing kernel rather than the Full-Width at Half-Max (FWHM) in mm. 
    * [Link](https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=FSL;d7249c17.1301) to an informative post on the FSL listserv
    * Conversion from sigma to FWHM = `sigma*sqrt(8*ln(2)) = sigma*2.3548`
    * Conversion from FWHM to sigma = `mm/sqrt(8*ln(2))`

In [ ]:
# Usage 
susan

In [7]:
# Strip the skull from the motion corrected and possibly slice time corrected image

# Take mean of the functional image over time
fslmaths my_bold_mcf_st.nii.gz -Tmean mean_func
# Bet the mean func, tell it to only generate the mask for now, then we rename mask_mask to mask
bet2 mean_func mask -f .3 -n -m; mv mask_mask.nii.gz mask.nii.gz
# Apply mask to create brain extracted functional image
fslmaths my_bold_mcf_st.nii.gz -mas mask.nii.gz my_bold_mcf_st_bet.nii.gz

In [3]:
# Take a look
fslview my_bold_mcf_st_bet.nii.gz mask.nii.gz

In [9]:
# FSL preprocessing steps to find brightness thresh and create liberal functional mask 
robust_range=$(fslstats my_bold_mcf_st_bet.nii.gz -p 2 -p 98)

# Find "robust" range from 2nd to 98th percentile
# Take 10th percentile of this robust range and threshold functional image and make new mask
mask_thresh=$(echo "scale=1; var1=$(echo ${robust_range} | awk -F' ' '{print $2}'); var2=var1/10; var2" | bc)

# Voxels most likely noise get excluded from this mask
fslmaths my_bold_mcf_st_bet.nii.gz -thr ${mask_thresh} -Tmin -bin mask -odt char

# Dilate the mask to give some extra buffer space
fslmaths mask -dilF mask

# Find the median within this mask and then 75% of the median within that mask 
brightness_thresh=$(fslstats my_bold_mcf_st.nii.gz -k mask -p 50 | awk '{print ($1*0.75)}')

# Apply mask and create new mean func
fslmaths my_bold_mcf_st.nii.gz -mas mask.nii.gz my_bold_mcf_st_thresh.nii.gz
fslmaths my_bold_mcf_st_thresh.nii.gz -Tmean mean_func


robust range is 0.000000 1113.125000 
mask thresh is 111.3


In [10]:
# Now we're ready to smooth!

# If working from the terminal, convert desired FWHM (mm) to sigma for input to SUSAN
# Below is an example of smoothing at 6mm FWHM
smooth=6
smoothSigma=$(echo $smooth | awk '{print ($1/(sqrt(8*log(2))))}')
echo "Smoothing kernel set to ${smooth}mm which is ${smoothSigma} in sigma"

Smoothing kernel set to 6mm which is 2.54797 in sigma


In [11]:
# Call to SUSAN
susan my_bold_mcf_st_thresh.nii.gz \
      ${brightness_thresh} ${smoothSigma} \
      3 1 1 \
      mean_func.nii.gz \
      ${brightness_thresh} \
      my_bold_mcf_st_thresh_smooth-6mm.nii.gz


In [12]:
# Compare un-smoothed and smoothed functional image
fslview my_bold_mcf_st_thresh.nii.gz my_bold_mcf_st_thresh_smooth-6mm.nii.gz

## ICA-AROMA

* In order to run ICA-AROMA you need:
    * preprocessed functional image up to smoothing: my_bold_mcf_st_thresh_smooth.nii.gz
    * .par file of motion parameters
    * registration files for pushing the functional to standard MNI 152 space
    * a more snug mask with our mean func as suggested in manual

In [ ]:
git clone https://github.com/maartenmennes/ICA-AROMA.git

In [13]:
# Make a snugger mask
bet mean_func.nii.gz aroma_mask -f .3 -n -m

In [ ]:
fslview mean_func.nii.gz aroma_mask_mask.nii.gz

In [15]:
# Call to ICA-AROMA
# On MV VM this took about 15-20 mins

# python ICA-AROMA/ICA_AROMA.py \
#   -in ../my_bold_mcf_st_thresh_smooth-6mm.nii.gz \
#   -out ../ica_aroma_6mm_test \
#   -affmat ../func2T1.mat \
#   -warp ../T1_2_MNI_warp.nii.gz \
#   -mc ../my_bold_mcf.par \
#   -m ../aroma_mask_mask.nii.gz


------------------------------- RUNNING ICA-AROMA ------------------------------- 
--------------- 'ICA-based Automatic Removal Of Motion Artifacts' --------------- 

Step 1) MELODIC
Step 2) Automatic classification of the components
  - registering the spatial maps to MNI
  - extracting the CSF & Edge fraction features
  - extracting the Maximum RP correlation feature
  - extracting the High-frequency content feature
  - classification
Step 3) Data denoising

----------------------------------- Finished -----------------------------------



In [22]:
hp_freq=0.008 
lp_freq=0.08
TR=2
hp_sigma=$(echo "1/(2*${TR}*${hp_freq})" | bc -l)
lp_sigma=$(echo "1/(2*${TR}*${lp_freq})" | bc -l)
echo "high pass in Hz $hp_freq"
echo "high pass in sigma $hp_sigma"
echo "low pass in Hz $lp_freq"
echo "low pass in sigma $lp_sigma"

high pass in Hz 0.008
high pass in sigma 31.25000000000000000000
low pass in Hz 0.08
low pass in sigma 3.12500000000000000000


In [ ]:
fslmaths | grep -- "-bptf"

In [ ]:
fslmaths ica_aroma_6mm/denoised_func_data_nonaggr.nii.gz \
         -bptf ${hp_sigma} ${lp_sigma} \
         my_bold_mcf_st_thresh_smooth-6mm_tempfilt-fsl.nii.gz

In [ ]:
3dBandpass -prefix ica_aroma_6mm/denoised_func_data_nonaggr.nii.gz  \
           ${hp_freq} ${lp_freq} \
           my_bold_mcf_st_thresh_smooth-6mm_tempfilt-afni.nii.gz

In [ ]:
# FSL power spectrum
fslpspec my_bold_mcf_st_thresh_smooth-6mm_tempfilt-fsl.nii.gz pwr_spct_fsl.nii.gz
# get a summary measure
fslmeants -i pwr_spct_fsl.nii.gz -m aroma_mask_mask.nii.gz

# AFNI power spectrum
fslpspec my_bold_mcf_st_thresh_smooth-6mm_tempfilt-afni.nii.gz pwr_spct_afni.nii.gz
# get a summary measure
fslmeants -i pwr_spct_afni.nii.gz -m aroma_mask_mask.nii.gz